In [ ]:
from aocd import data, models, submit
from pathlib import Path
import re

import numpy as np
from itertools import product

# Load data and examples

In [ ]:
puzzle_year = 2024
puzzle_day = int(re.match(r"day(\d+)", Path.cwd().name).group(1))

In [ ]:
todays_puzzle = models.Puzzle(year=puzzle_year, day=puzzle_day)
todays_examples = todays_puzzle.examples

In [ ]:
todays_examples[0] = todays_examples[0]._replace(
    input_data="""RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE
"""
)

# Part A

In [ ]:
def single_plant_plot_cost(char_map, visited, i, j):
    n = char_map.shape[0]

    def single_plant_plot_area_perimeter(x, y):
        if visited[(x, y)]:
            return (0, 0)
        visited[(x, y)] = True
        point_char = char_map[(x, y)]
        area = 1
        perimeter = 0
        for d_x, d_y in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            next_x, next_y = x + d_x, y + d_y
            sub_area = 0
            sub_perimeter = 1
            if 0 <= next_x < n and 0 <= next_y < n:
                if char_map[(next_x, next_y)] == point_char:
                    sub_area, sub_perimeter = single_plant_plot_area_perimeter(
                        next_x, next_y
                    )
            area += sub_area
            perimeter += sub_perimeter
        return area, perimeter

    area, perimeter = single_plant_plot_area_perimeter(i, j)
    return area * perimeter

In [ ]:
def part_a(data: str) -> str:
    char_map = np.array([[c for c in line] for line in data.split()])
    visited = np.zeros_like(char_map)
    n = char_map.shape[0]
    result = 0
    for i in range(n):
        for j in range(n):
            if not visited[(i, j)]:
                result += single_plant_plot_cost(char_map, visited, i, j)
    return str(result)

In [ ]:
for example_index, example in enumerate(todays_examples):
    if example.answer_a != "":
        print(
            f"Example {example_index} part a: {part_a(example.input_data)} (expected {example.answer_a})"
        )
        assert part_a(str(example.input_data)) == example.answer_a
submit(part_a(data), part="a", year=puzzle_year, day=puzzle_day)

# Part B

In [ ]:
todays_examples[0] = todays_examples[0]._replace(answer_b="1206")

In [ ]:
def safe_get(array: np.ndarray, index, default):
    n = array.shape[0]
    if 0 <= index[0] < n and 0 <= index[1] < n:
        return array[tuple(index)]
    else:
        return default


def single_plant_plot_cost_part_b(char_map, visited, i, j):
    n = char_map.shape[0]

    def is_edge_in_direction(curr_position: tuple, direction: tuple):
        first_neighbour = safe_get(
            char_map, (curr_position[0] + direction[0], curr_position[1]), "."
        )
        second_neighbour = safe_get(
            char_map, (curr_position[0], curr_position[1] + direction[1]), "."
        )
        diagonal_meighbour = safe_get(
            char_map,
            (curr_position[0] + direction[0], curr_position[1] + direction[1]),
            ".",
        )
        curr_char = char_map[curr_position]
        if (
            curr_char == first_neighbour
            and curr_char == second_neighbour
            and curr_char != diagonal_meighbour
        ):
            return True
        if curr_char != first_neighbour and curr_char != second_neighbour:
            return True
        return False

    def single_plant_plot_area_edges(x, y):
        if visited[(x, y)]:
            return (0, 0)
        visited[(x, y)] = True
        point_char = char_map[(x, y)]
        area = 1
        edges = sum(
            [
                is_edge_in_direction((x, y), direction)
                for direction in product([-1, 1], repeat=2)
            ]
        )

        for d_x, d_y in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            next_x, next_y = x + d_x, y + d_y
            sub_area = 0
            sub_edges = 0
            if 0 <= next_x < n and 0 <= next_y < n:
                if char_map[(next_x, next_y)] == point_char:
                    sub_area, sub_edges = single_plant_plot_area_edges(next_x, next_y)
            area += sub_area
            edges += sub_edges
        return area, edges

    area, edges = single_plant_plot_area_edges(i, j)
    return area * edges

In [ ]:
def part_b(data: str) -> str:
    char_map = np.array([[c for c in line] for line in data.split()])
    visited = np.zeros_like(char_map)
    n = char_map.shape[0]
    result = 0
    for i in range(n):
        for j in range(n):
            if not visited[(i, j)]:
                result += single_plant_plot_cost_part_b(char_map, visited, i, j)
    return str(result)

In [ ]:
for example_index, example in enumerate(todays_examples):
    if example.answer_b != "":
        print(
            f"Example {example_index} part b: {part_b(example.input_data)} (expected {example.answer_b})"
        )
        assert part_b(str(example.input_data)) == example.answer_b
submit(part_b(data), part="b", year=puzzle_year, day=puzzle_day)